|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating neurons and dimensions<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Negation tuning in MLP neurons<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec

import statsmodels.api as sm

import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, GPT2Tokenizer

import requests

# vector plots
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Exercise 1: Import the model and implant MLP hooks

In [ ]:
# load GPT2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained('gpt2-large')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# move the model to the GPU


In [ ]:
# this time, hook the output instead of re-calculating the linear layer
activations =

def implant_hook(layer_number):
  def hook(module, input, output):

    # store in the dictionary
    activations
  return hook

# put hooks in all layers


In [ ]:
nneurons =

# Exercise 2: Get text, find negations and affirmations

In [ ]:
# https://gutenberg.org/ebooks/32154
text = requests.get('https://gutenberg.org/cache/epub/32154/pg32154.txt').text
tokens = tokenizer.encode(text,return_tensors='pt')
num_tokens =
print(f'There are {num_tokens:,} tokens, of which {} are unique.')

In [ ]:
# context window size (tokens before and after each target)
context_pre =
context_pst =

In [ ]:
# list of negation words to match exactly
negation_words = []

# initialize vector
isNegation = np.zeros(num_tokens,dtype=)

# loop over all tokens
for ti in

  # current token
  currtok = tokenizer.decode

  # token contains a 't contraction
  condA =

  # word need to match completely (c.f., not->noted, nor->enormous)
  condB =

  # next token starts with a space (c.f., not->connotative)
  condC =

  # test
  if ...:
    isNegation[ti] = 1


# count the targets
numNegationTokens =

In [ ]:
negationsIdx =
context_win = 15

# examine some proper nouns
print(f'There are {numNegationTokens} "negation" tokens in the dataset.\n')
for t in negationsIdx[:10]:
  print(f'Example {t}:

In [ ]:
# all the negation tokens
tokenizer.decode(tokens[0,negationsIdx])

In [ ]:
# list of affirmation words to match exactly
affirmation_words = []

# initialize vector
isAffirmation = np.zeros

# loop over all tokens
for ti in

  # current token
  currtok =

  # next token can't be 'not'
  condA =

  # word need to match completely (c.f., not->noted, nor->enormous)
  condB =

  # next token starts with a space (c.f., not->connotative)
  condC =

  # test
  if
    isAffirmation[ti] = 1

# count the number of target tokens
numAffirmationTokens =

In [ ]:
affirmationsIdx =

# examine some examples
print(f'There are {numAffirmationTokens} "affirmation" tokens in the dataset.\n')
for t in affirmationsIdx[:10]:
  print(

# Exercise 3: Create batches and get activations

In [ ]:
# create batches
batch_negations = torch.zeros((),dtype=)
batch_affirmations = torch.zeros


# negation sequences
for b in range(numNegationTokens):
  tokenLoc =
  batch_negations[b,:] = tokens

# affirmation sequences
for b in range(numAffirmationTokens):

  batch_affirmations

#
print('Shape of negations batch:',batch_negations.shape)
print('Shape of affirmations batch:',batch_affirmations.shape)

In [ ]:
# process the target (negation) tokens
with torch.no_grad():
  model(batch_negations.to(device))

# copy the activations



### repeat for affirmations tokens


In [ ]:
print(affirmations_activations.keys(),'\n')

affirmations_activations['mlp_5'].shape

# Exercise 4: Logistic regression in all neurons from one layer

In [ ]:
# a vector of category labels (0's for affirmation words, 1's for negation words)
category_labels =

In [ ]:
# MLP transformer layer
whichLayer2use = 13

In [ ]:
# confirm getting the right token
tokenizer.decode(batch_negations[3,context_pre]), tokenizer.decode(batch_affirmations[3,context_pre])

In [ ]:
# initialize matrix to store the classifier results
classifierResults = np.full(size,np.nan)

# loop over neurons for per-neuron analysis
for neuroni in

  # vectorize the activations over batches
  targs = negations_activations
  comps = affirmations_activations

  # build and run the model
  try:
    result = sm.Logit(
        ,

        ).fit(maxiter=3000,disp=0)

    # extract the results (p-value and beta)
    classifierResults[neuroni,0] = # p-value of beta1
    classifierResults[neuroni,1] = # beta1
  except: pass

In [ ]:
# visualization of model significance and sign

# setup the figure
fig = plt.figure(figsize=(12,4))
gs = GridSpec(1,4,figure=fig)

ax0 = fig.add_subplot(gs[:3])
ax1 = fig.add_subplot(gs[3])

# find the negative and positive betas, and the supra-threshold results
negBetas =
posBetas =
pvalThresh =  # p<.05, Bonferroni-corrected
sigBetas =


# positive significant betas
idx2plot = posBetas & sigBetas
ax0.plot(np.where()[0],classifierResults[],'ro',markerfacecolor=[.7,.7,.7],label='Positive and sig.')
ax1.plot(classifierResults[idx2plot,1],,'ro',markerfacecolor=[.7,.7,.7,.5])

# positive non-significant betas


# negative significant betas


# negative non-significant betas


ax0.set(ylabel='Beta coefficient',xlabel='Neuron index',xlim=[-10,nneurons+9],
              title='Statistical parameters of negation-term classification')
ax0.legend(fontsize=8)


ax1.axhline(-np.log(pvalThresh),linestyle='--',color='b',label='Significance threshold')
ax1.set(xlabel='Beta coeff',ylabel='-log(p)',title='Betas by p-values')
ax1.legend(fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# find the neuron with best classification
maxBeta = np.max(
maxBetaNeuron = np.where
maxBetaNeuron

In [ ]:
# show the prediction probabilities for the max neuron

# need to re-run the model for that neuron
targs = negations_activations[f'mlp_{whichLayer2use}']
comps = affirmations_activations
result = sm.Logit(,sm.add_constant).fit

# per-token accuracy
accuracy =

In [ ]:
# visualization
plt.figure(figsize=(10,4))

plt.plot(category_labels,'rx')
plt.plot(result.predict(),'ko',markerfacecolor=[.7,.9,.7,.5],markersize=8)


plt.show()

# Exercise 5: Text heatmap of “best” neuron’s activations

In [ ]:
# scale the activity for colormapping
negActs = negations_activations
negActsNorm =

In [ ]:
# all of this code is copied from several previous code files
fig,ax = plt.subplots(figsize=(10,2))
temp_text = ax.text(0,0,'n',fontsize=12,fontfamily='monospace')
bbox = temp_text.get_window_extent(renderer=fig.canvas.get_renderer())
inv = ax.transAxes.inverted()
bbox_axes = inv.transform([[bbox.x0,bbox.y0], [bbox.x1,bbox.y1]])
en_width = bbox_axes[1,0] - bbox_axes[0,0] # bbox is [(x0,y0),(x1,y1)]
plt.close(fig)

In [ ]:
x_pos = 0  # starting x position (in axis coordinates)
y_pos = 1  # vertical center

fig, ax = plt.subplots(figsize=(10,2))
ax.axis('off')


# loop over rows in the batch
for batchi in range(20): # just the first 20 sequences

  for toki in range(context_pre-5,batch_negations.shape[1]):

    # text of this token
    toktext = tokenizer.decode([batch_negations[batchi,toki]])

    # width of the token
    token_width = en_width*len(toktext)

    # text object with background color matching the activation
    ax.text(x_pos+token_width/2, y_pos, toktext, fontsize=12, ha='center', va='center',fontfamily='monospace',
            bbox = dict(boxstyle='round,pad=.3', facecolor=mpl.cm.Reds(negActsNorm[batchi,toki]**2), edgecolor='none', alpha=.8))

    # update the token counter and x_pos
    x_pos += token_width + .01 # plus a small gap

  # end of the line; reset coordinates and counter
  y_pos -= .17
  x_pos = 0

plt.show()

# Exercise 6: Laminar profile of classification

In [ ]:
# initialize matrix to store the classifier results
pvalues  =  # initialize to 1's to ignore in subsequent mask
betas    = np.zeros
accuracy =


# loop over layers
for layeri

  # loop over neurons for per-neuron analysis
  for neuroni
    # vectorize the activations over batches
    targs = negations_activations
    comps = affirmations_activations

    # build and run the model
    try: # sometimes crashes for linear-algebra reasons
      result =

      # extract the results (p-value, beta, and accuracy)
      pvalues[layeri,neuroni]  =
      betas[layeri,neuroni]    =
      accuracy[layeri,neuroni] =

    except: pass

  print(f'Finished layer {layeri+1:2}/{model.config.n_layer}')

In [ ]:
# create two masks
pvalue_mask =
posbet_mask =

# get accuracy only from masked neurons
# gratuitously confusingly, np.ma.masked_where() actually keeps the False values, and masks *out* True values
masked_accuracy = np.ma.masked_where

# make the plot
_,axs = plt.subplots(1,2,figsize=(12,4))

axs[0].plot(,'kH',markerfacecolor=[.7,.7,.7],markersize=9)
axs[0].set(xlabel='Layer',ylabel='Percent significant tests (%)',title='Laminar profile of significance')

axs[1].plot(,'ko',markerfacecolor=[.9,.7,.7],markersize=9,label='All tests')
axs[1].plot(,'ks',markerfacecolor=[.7,.9,.7],markersize=9,label='Only sig. $\\beta$s>0')
axs[1].legend()

axs[1].set(xlabel='Layer',ylabel='Prediction accuracy (%)',title='Laminar profile of prediction accuracy')
plt.show()